### Модуль 2. Спам не пройдет
Задание: Написать классификатор спама с использованием теоремы Байеса.

Автор: Руслан Когочкин (https://www.kaggle.com/ruslankogochkin)

In [1]:
# Внимание! Функция train() может выполняться до 5 минут. 

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords # pip install nltk

pA = 0  
pNotA = 0  
spam = {}
not_spam = {}
number_spam_words = 0
number_not_spam_words = 0
uniqueWordsTrainSet = set()
url_data = 'https://raw.githubusercontent.com/rand89/data_science1/master/data/spam_or_not_spam.csv'

def get_train_data():
    """ Считывает данные из csv-файла и возвращает список train_data"""
    train_data = []
    df = pd.read_csv(url_data)
    df = df.dropna()
    for i in df.index:
        train_data += [df.loc[i]]
    return train_data


def get_train_data_test():
    """ Считывает данные из X_train, y_train и возвращает список train_data"""
    train_data = []
    df_train = pd.DataFrame({'email': X_train, 'label': y_train})
    for i in df_train.index:
        train_data += [df_train.loc[i]]
    return train_data


def prepare_text(text):
    """ Принимает текст text, заменяет числа, удаляет знаки препинания,
    удаляет стоп-слова и возвращает список, состоящий из очищенных слов."""
    words = []
    text = text.lower()
    text = re.sub('[0-9]+', 'number', text)
    text = re.sub('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', ' ', text)
    text = re.sub(' +', ' ', text).strip()
    for word in text.split(' '):
        if word not in stopwords.words('english'): words.append(word)    
    return words


def calculate_word_frequencies(body, label):
    """ Принимает текст письма body и метку label (1/0).
    Добавляет слова в множество уникальных слов uniqueWordsTrainSet.
    Записывает слова письма в словари spam и not_spam.
    Считает количество number_spam_words и number_not_spam_words."""
    global spam, not_spam, number_spam_words, number_not_spam_words, uniqueWordsTrainSet
    for word in prepare_text(body):
        uniqueWordsTrainSet.add(word)
        if label == 1:
            number_spam_words += 1
            spam[word] = spam.get(word, 0) + 1
        else:
            number_not_spam_words += 1
            not_spam[word] = not_spam.get(word, 0) + 1

                
def train():
    """ Для каждого письма в тренировочном наборе train_data
    вызывает функцию calculate_word_frequencies.
    Считает количество number_spam_emails и number_not_spam_emails.
    Сохраняет логарифмы вероятностей pA и pNotA."""
    global pA, pNotA
    number_spam_emails = 0
    number_not_spam_emails = 0
    # train_data = get_train_data()
    train_data = get_train_data_test()
    for email in train_data:
        calculate_word_frequencies(email[0], email[1])
        if email[1] == 1:
            number_spam_emails += 1
        else:
            number_not_spam_emails += 1  
    pA = np.log(number_spam_emails / (number_spam_emails + number_not_spam_emails))
    pNotA = np.log(number_not_spam_emails / (number_spam_emails + number_not_spam_emails))

    
def calculate_P_Bi_A(word, label):
    """ Принимает слово word и метку label (1/0).
    Возвращает логарифм вычисленной вероятности P_Bi_A со сглаживанием Лапласа."""
    if label == 1:
        P_Bi_A = (spam.get(word, 0) + 1) / (number_spam_words + len(uniqueWordsTrainSet))
    else:
        P_Bi_A = (not_spam.get(word, 0) + 1) / (number_not_spam_words + len(uniqueWordsTrainSet))    
    return np.log(P_Bi_A)


def calculate_P_B_A(text, label):
    """ Принимает текст письма text и метку label (1/0).
    Возвращает сумму результатов выполнения функции calculate_P_Bi_A."""
    P_B_A = 0
    for word in prepare_text(text):
        P_B_A += calculate_P_Bi_A(word, label)
    return P_B_A


def classify(email):
    """ Принимает текст письма email. Вычисляет вероятности P_A_B и P_notA_B.
    Возвращает результат сравнения P_A_B и P_notA_B."""
    P_A_B = pA + calculate_P_B_A(email, 1)
    P_notA_B = pNotA + calculate_P_B_A(email, 0)
    if P_A_B > P_notA_B:
        return True
    else:
        return False

### Обучение

In [2]:
df = pd.read_csv(url_data)
df = df.dropna()

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['email'], df['label'], test_size=0.25, random_state=42)

In [4]:
train()

### Тест

In [5]:
y_pred = []
for i in range(len(X_test)):
        y_pred.append(classify(X_test.iloc[i]))

In [7]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

print('accuracy_score:', accuracy_score(y_test, y_pred))
print('f1_score:', f1_score(y_test, y_pred))
print('precision_score:', precision_score(y_test, y_pred))
print('recall_score:', recall_score(y_test, y_pred))

accuracy_score: 0.9933333333333333
f1_score: 0.979253112033195
precision_score: 1.0
recall_score: 0.959349593495935


In [8]:
confusion_matrix(y_test, y_pred)

array([[627,   0],
       [  5, 118]], dtype=int64)

### Выводы
Классификатор спама показал отличные результаты на тестовой выборке. Он совершенно точно определяет, что письмо является спамом (precision_score = 1), но обнаруживает только 96% спама (recall_score = 0.959). 5 спам-писем классификатор отнес к нормальным сообщениям (см. confusion_matrix). F1_score = 0.979, что является прекрасным показателем.